In [1]:
import sys
import datetime
import pandas as pd
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver

In [2]:
#Not implemented
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

In [3]:
#Not implemented
cubes = ['222', '333', '444', '555', '666', '777']

In [4]:
opt = Options()
opt.add_experimental_option('prefs', {'intl.accept_languages': 'en,en_US'})

In [5]:
driver = webdriver.Edge(service = Service(EdgeChromiumDriverManager().install()), options = opt)

In [6]:
competitions = 'https://www.worldcubeassociation.org/competitions?event_ids%5B%5D=333&event_ids%5B%5D=222&event_ids%5B%5D=444&event_ids%5B%5D=555&event_ids%5B%5D=666&event_ids%5B%5D=777&region=all&search=&year=all+years&state=past&from_date=&to_date=&delegate=&display=list'

In [7]:
driver.get(competitions)

In [8]:
competitions_we = driver.find_elements(By.XPATH, '//li [@class = "list-group-item past"]')

In [9]:
def date_parse (x : str):
    (x1 := x.split(','))
    if not x.find(' - '): return datetime.datetime.strptime(x, '%b %d, %Y').date()
    elif x.count(',') -1: return datetime.datetime.strptime((x.split(' - ')[0]), '%b %d, %Y').date()
    else: return datetime.datetime.strptime((f'{x1[0].split(" - ")[0]},' + x1[1]), '%b %d, %Y').date()

In [10]:
country_parse = lambda x: [(x1 := x.split(', '))[0], x1[-1]]

In [11]:
competition_catalog = {}
init = datetime.datetime.now()
for i, competition in enumerate(competitions_we):
    element = competition.find_element(By.CLASS_NAME, 'competition-link')
    u = 50/(e := len(competitions_we))
    competition_catalog[element.text] = {
        'url': element.find_element(By.TAG_NAME, 'a').get_attribute('href'),
        'country': country_parse(
            competition.find_element(By.CLASS_NAME, 'location').text)[0],
        'city/state': country_parse(
            competition.find_element(By.CLASS_NAME, 'location').text)[1],        
        'date': date_parse(competition.find_element(By.CLASS_NAME, 'date').text)}
    sys.stdout.write('\r')
    sys.stdout.write(
        "[{n}{s}] {p}% Elapsed time: {t}s".format(
            n = '='*round(i*u),
            s = ' '*round((e-i)*u),
            p = round((i/e)*100),
            t = (datetime.datetime.now() - init)))
    sys.stdout.flush()

[==================================================] 100% Elapsed time: 0:08:52.066824s

In [12]:
driver.quit()